## Step 1 Loading independices

In [ ]:
import tensorflow as tf # Call tensorflow which is used for running datasets, construct, train and evaluation results

In [ ]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

## Step 2 Loading  3 data sets, Training, Test, Validation

In [4]:
print("MNIST training number of training datasets:", len(mnist.train.images))
print("MNIST training number of labels:", len(mnist.train.labels))
print("MNIST validation number of datasets:", len(mnist.validation.images))
print("MNIST validation number of labels:", len(mnist.validation.labels))
print("MNIST test number of training data:", len(mnist.test.images))
print("MNIST test number of labels:", len(mnist.test.labels))
print("MNIST training data set pixel size", (mnist.test.images[0].size))

MNIST training number of training datasets: 55000
MNIST training number of labels: 55000
MNIST validation number of datasets: 5000
MNIST validation number of labels: 5000
MNIST test number of training data: 10000
MNIST test number of labels: 10000
MNIST training data set pixel size 784


## Step 3 Initialize placeholder and variables

In [5]:
x = tf.placeholder(tf.float32, [None, 784]) # create 2d tensor of floating points, None denotes batch of any size
                                            # 784 denotes dimensionality, 28 by 28 pixel 
W = tf.Variable(tf.zeros([784, 10])) # initialize weights variables to zero, 784 features and 10 outputs
b = tf.Variable(tf.zeros([10])) # initialize bias varibale to zero, 10 classess

y = tf.nn.softmax(tf.matmul(x, W) + b)  # Implement the model, multiply the vectorized input images x by the weight matrix W, add the bias b
y_ = tf.placeholder(tf.float32, [None, 10]) # Intitate placeholder for loss function

def weight_variable(shape):
  initial = tf.truncated_normal(shape, stddev=0.1)
  return tf.Variable(initial)

def bias_variable(shape):
  initial = tf.constant(0.1, shape=shape)
  return tf.Variable(initial)

def conv2d(x, W):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

## Step 4 Construct a convolution neural network

In [6]:
import tensorflow as tf
sess = tf.InteractiveSession()

W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])
x_image = tf.reshape(x, [-1,28,28,1])
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)
W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

y_conv = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

##  Step 5 Train constructed convolution model

In [7]:
cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_conv))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
sess.run(tf.global_variables_initializer())
for i in range(200):
  batch = mnist.train.next_batch(50)
  if i%100 == 0:
    train_accuracy = accuracy.eval(feed_dict={
        x:batch[0], y_: batch[1], keep_prob: 1.0})
    print("step %d, training accuracy %g"%(i, train_accuracy))
  train_step.run(feed_dict={x: batch[0], y_: batch[1], keep_prob: 0.5})

step 0, training accuracy 0.08
step 100, training accuracy 0.84


## Step 6 Evaluate results

In [8]:
print("test accuracy %g"%accuracy.eval(feed_dict={
    x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))

test accuracy 0.9039
